In [1]:
import json
from pathlib import Path

import pandas as pd
import requests

from config import Config

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:,.2f}".format)

In [3]:
base_dir = Path(".").resolve()
data_dir = base_dir / "data"

In [4]:
url = "https://distributor-rules.logitravelgroup.com/api/organizations/lgt/agencies/products/hot/level_commissions/new"
headers = {
    "Content-Type": "application/json",
    "Accept": "text/plain",
    "Authorization": f"ApiKey {Config.APIKEY}",
}

In [5]:
csv_file = data_dir / "pricing_rules_20250123.csv"

In [6]:
df = pd.read_csv(csv_file)
# df["remarks"] = (
#     df["remarks"] + "_Override_Push_2025 | " + df["commission"].astype(str) + "%"
# )
display(df)

,credential_id,credential_name,client_id,client_name,dynamic_commission,action
0,34060,Planet travel,1775,Planet Travel LATAM,True,0.20
1,35005,PLANET TRAVEL API,1775,Planet Travel LATAM,True,0.20
2,34055,Ricale Viajes,1774,Ricale Viajes,True,0.20
3,33958,Organización Piamonte,1753,Organización Piamonte,True,0.20
4,33833,Agenda Events and Conferences LLC,1716,Agenda Events and Conferences LLC,False,-0.50
5,34179,TURIVEL (ES),1698,Turivel,True,-0.50
6,33589,Dida Travel B2B|TPS Singapore,1696,DIDATRAVEL TECHNOLOGY SINGAPORE,True,-0.50
7,33591,Dida Travel B2B|DIR Singapore,1696,DIDATRAVEL TECHNOLOGY SINGAPORE,True,-0.50
8,33592,Dida Travel Sensitive Retail Singapore,1696,DIDATRAVEL TECHNOLOGY SINGAPORE,True,-0.50
9,33491,3 Full Steps,1671,3 FULL STEPS,True,-0.50


In [7]:
df.dtypes

credential_id           int64
credential_name        object
client_id               int64
client_name            object
dynamic_commission       bool
action                float64
dtype: object

In [8]:
# df[["bod_from", "bod_to", "cid_from", "cid_to"]] = df[
#     ["bod_from", "bod_to", "cid_from", "cid_to"]
# ].apply(pd.to_datetime, errors="coerce", format="%d/%m/%Y")

In [9]:
df_grouped = (
    df.groupby(
        [
            "client_id",
            "client_name",
            "action",
            "dynamic_commission",
        ],
        dropna=False,
    )
    .agg({"credential_id": lambda x: list(set(map(str, x)))})
    .reset_index()
)

df_grouped["label"] = df_grouped.apply(
    lambda row: f"February Push - {row['client_name']}",
    axis=1,
)

df_mgr = df_grouped[df_grouped.client_id.isin([380, 1662])]
df_lgt = df_grouped[~df_grouped.client_id.isin([380, 1662])]

In [10]:
display(df_mgr)

,client_id,client_name,action,dynamic_commission,credential_id,label
12,380,The Flights Guru,-0.50,True,[6480],February Push - The Flights Guru
27,1662,Via Holiday,-0.50,True,[33407],February Push - Via Holiday


In [11]:
display(df_lgt)

,client_id,client_name,action,dynamic_commission,credential_id,label
0,131,Methabook,-0.50,False,"[9021, 4162, 11365]",February Push - Methabook
1,132,Rezlive,0.20,True,"[34366, 4770]",February Push - Rezlive
2,139,Escalabeds,0.30,True,"[33903, 33904, 34083]",February Push - Escalabeds
3,146,Central de Reservas,0.20,True,"[33637, 34127, 21216, 3693, 33539]",February Push - Central de Reservas
4,157,All Seasons,0.20,True,"[33875, 35003]",February Push - All Seasons
5,186,Nuitee,0.10,True,"[4396, 8733, 4982, 35000]",February Push - Nuitee
6,211,Holisto (Splitty-Traveluro),-0.50,True,"[11986, 11994, 11992, 11995, 11993]",February Push - Holisto (Splitty-Traveluro)
7,235,6 Tour,0.30,True,[5060],February Push - 6 Tour
8,312,AIRTOUR,0.20,True,[6016],February Push - AIRTOUR
9,315,Tale Travel,-0.50,True,[32834],February Push - Tale Travel


In [12]:
# df_grouped = df_grouped.replace({pd.NaT: None, pd.NA: None})

In [13]:
def get_date_range(date_values=None):
    if date_values is None:
        return {"t": 0}

    from_date, to_date = date_values
    if from_date is None or to_date is None:
        return {"t": 0}
    else:
        return {
            "t": 1,
            "f": from_date.strftime("%Y-%m-%d"),
            "u": to_date.strftime("%Y-%m-%d"),
        }

In [14]:
results_list = []
total_rows = len(df_lgt)

for index, row in df_lgt.iterrows():
    data = {
        "id": None,
        "name": row["label"],
        "description": row["label"],
        "com": row["action"],
        "lvl": {
            "cli": {"t": 1, "l": row["credential_id"]},
            "cp": 0,
            "rat": 0,
            "prv": {"t": 0, "l": None},
            "hot": {"t": 0, "l": None},
            "mrk": {"t": 0, "l": None},
            "mel": {"t": 0, "l": None},
            "cid": {"t": 1, "f": "2025-02-01", "u": "2025-02-28"},
            "bod": {"t": 1, "f": "2025-01-24", "u": "2025-01-30"},
            "rel": 0,
            "dow": {"t": 0, "l": None},
            "age": 0,
            "room": {"t": 0, "l": None},
            "non": {"t": 0, "l": None},
            "hou": {"t": 0, "l": None},
            "isDynamicCommission": row["dynamic_commission"],
            "ovr": False,
            "prg": {"t": 0, "f": 0, "u": 0},
            "ps": {"t": 0, "l": None},
        },
    }

    print(f"Processing row {index + 1}/{total_rows}")
    # print(json.dumps(data, indent=2))
    # break
    try:
        response = requests.put(url, headers=headers, json=data)

        if response.status_code == 200:
            data = response.json()["result"]
            data.update(data.pop("lvl"))

            results_list.append(data)

        elif response.status_code == 404:
            print("Error: Agencies not found.")
        else:
            print(
                f"Error: Received unexpected status code {response.status_code}. Details: {response.text}"
            )

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")

Processing row 1/33
Processing row 2/33
Processing row 3/33
Processing row 4/33
Processing row 5/33
Processing row 6/33
Processing row 7/33
Processing row 8/33
Processing row 9/33
Processing row 10/33
Processing row 11/33
Processing row 12/33
Processing row 14/33
Processing row 15/33
Processing row 16/33
Processing row 17/33
Processing row 18/33
Processing row 19/33
Processing row 20/33
Processing row 21/33
Processing row 22/33
Processing row 23/33
Processing row 24/33
Processing row 25/33
Processing row 26/33
Processing row 27/33
Processing row 29/33
Processing row 30/33
Processing row 31/33
Processing row 32/33
Processing row 33/33
Processing row 34/33
Processing row 35/33


In [15]:
metadata = {
    "editState": "edit_state",
    "isObsolete": "obsolete",
    "name": "name",
    "description": "description",
    "lastUser": "last_user",
    "lastDate": "last_date",
    "com": "commission",
    "id": "id",
    "rrg": "release_range",
    "prg": "price_range",
    "cli": "credential",
    "ps": "paying_society",
    "cp": "refundable",
    "rat": "rate",
    "prv": "provider",
    "hot": "hotel",
    "dest": "destination",
    "mrk": "market",
    "mel": "meal",
    "cid": "check_in",
    "bod": "booking_date",
    "rel": "max_release",
    "isDynamicCommission": "dynamic",
    "over": "override",
    "dow": "days_of_week",
    "age": "age",
    "room": "room",
    "non": "num_of_nights",
    "hou": "hours",
    "level_mapping": {"t": "level", "l": "list", "f": "from", "u": "to"},
}

In [16]:
for rule in results_list:
    # Rename keys according to metadata
    for key in list(rule.keys()):
        if key in metadata:
            rule[metadata[key]] = rule.pop(key)

    # Handle nested dictionary for level mapping
    level_mapping = metadata["level_mapping"]
    for key, value in rule.items():
        if isinstance(value, dict):
            for k in list(value.keys()):
                if k in level_mapping:
                    value[level_mapping[k]] = value.pop(k)

In [17]:
df_rules = pd.json_normalize(results_list, sep="_")

In [18]:
df_rules = df_rules[
    [
        "id",
        "last_user",
        "last_date",
        "name",
        "description",
        "credential_level",
        "credential_list",
        "rate",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "refundable",
        "meal_level",
        "meal_list",
        "market_level",
        "market_list",
        "paying_society_level",
        "paying_society_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "commission",
        "ovr",
        "dynamic",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "price_range_level",
        "price_range_from",
        "price_range_to",
        "release_range_level",
        "release_range_from",
        "release_range_to",
    ]
]

In [19]:
from datetime import datetime

df_rules.to_csv(
    f"pricing_rules_{datetime.today().strftime('%Y%m%d_%H%M%S')}.csv", index=False
)